# MemoryLLM: Self-Updatable Large Language Models 완전 가이드

이 노트북은 **MemoryLLM**의 Training과 Inference 코드를 상세히 분석하고 설명합니다.

---

## 논문 정보

| 항목 | 내용 |
|------|------|
| **제목** | MemoryLLM: Towards Self-Updatable Large Language Models |
| **학회** | ICML 2024 |
| **저자** | Yu Wang et al. |
| **GitHub** | [YuWangX/MemoryLLM](https://github.com/YuWangX/MemoryLLM) |
| **HuggingFace** | [YuWangX/memoryllm-7b](https://huggingface.co/YuWangX/memoryllm-7b) |

---

## 핵심 기여

MemoryLLM은 기존 LLM의 한계를 극복하기 위해 **Self-Updatable Memory Pool**을 도입했습니다:

1. **지식 업데이트 문제**: 기존 LLM은 학습 이후 새로운 지식을 반영하기 어려움
2. **긴 컨텍스트 처리**: 제한된 context window로 인한 정보 손실
3. **메모리 효율성**: 중요한 정보를 압축하여 저장하고 필요할 때 검색

### 핵심 아이디어

```
기존 LLM:     입력 → [Transformer Layers] → 출력
MemoryLLM:    입력 → [Memory Pool + Transformer Layers] → 출력
                         ↑
                    Self-Update
```

## 1.1 아키텍처 개요

MemoryLLM의 핵심 구조는 다음과 같습니다:

```
┌─────────────────────────────────────────────────────────────┐
│                      MemoryLLM                              │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   ┌─────────────────────────────────────────────────────┐   │
│   │              Memory Pool (Self-Updatable)           │   │
│   │   Shape: [L, num_blocks × num_tokens, hidden_size]  │   │
│   │   예시: [32, 50 × 256, 4096] = 1.67B parameters     │   │
│   └─────────────────────────────────────────────────────┘   │
│                         ↓ (concatenate)                     │
│   ┌─────────────────────────────────────────────────────┐   │
│   │            Transformer Decoder Layers               │   │
│   │      (LLaMA-2/3 기반, 32 layers, 4096 dim)          │   │
│   └─────────────────────────────────────────────────────┘   │
│                         ↓                                   │
│   ┌─────────────────────────────────────────────────────┐   │
│   │                    LM Head                          │   │
│   │              (Vocabulary Projection)                │   │
│   └─────────────────────────────────────────────────────┘   │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### 핵심 컴포넌트

| 컴포넌트 | 역할 | 크기 |
|---------|------|------|
| **Memory Pool** | 컨텍스트 정보 저장 | [32, 12800, 4096] |
| **BOS Embedding** | 각 레이어의 시작 토큰 | [32, 1, 4096] |
| **Positional Embedding** | 새 메모리 위치 인코딩 | [1, 1, 4096] |

## 1.2 메모리 업데이트 메커니즘

MemoryLLM의 핵심은 **inject_memory → update_memory** 사이클입니다:

```
┌──────────────┐     ┌──────────────┐     ┌──────────────┐
│   Context    │ --> │ inject_memory│ --> │ delta_memory │
│  (새 정보)    │     │    (주입)    │     │  (추출된 표현) │
└──────────────┘     └──────────────┘     └──────────────┘
                                                  │
                                                  ↓
┌──────────────┐     ┌──────────────┐     ┌──────────────┐
│ Memory Pool  │ <-- │update_memory │ <-- │ drop_memory  │
│   (갱신됨)   │     │   (저장)     │     │  (공간 확보)  │
└──────────────┘     └──────────────┘     └──────────────┘
```

### 단계별 설명

1. **inject_memory**: 새로운 context를 모델에 통과시켜 각 레이어의 hidden states 추출
2. **delta_memory**: 추출된 표현 (shape: `[batch, L, num_tokens, d]`)
3. **drop_memory**: 기존 메모리에서 1/num_blocks 만큼 랜덤 제거
4. **update_memory**: 새 delta_memory를 메모리 풀 끝에 추가

## 1.3 환경 설정

이 노트북을 실행하기 위해 필요한 환경을 설정합니다.

### 하드웨어 요구사항

| 모델 | GPU VRAM | 권장 GPU |
|------|----------|----------|
| memoryllm-7b | ~16GB | RTX 3090, A100 |
| memoryllm-8b | ~18GB | RTX 4090, A100 |
| Training | ~40GB+ | A100 80GB |

### 소프트웨어 버전

| 패키지 | 버전 |
|--------|------|
| Python | 3.10+ |
| PyTorch | 2.2-2.5 |
| Transformers | 4.40-4.48 |
| Flash Attention | 2.x |

In [ ]:
# Cell 4: 의존성 설치
# 이미 설치되어 있다면 이 셀을 건너뛰세요

# 기본 의존성
# !pip install torch>=2.2.0 transformers>=4.40.0

# 추가 의존성
# !pip install peft pytorch-lightning omegaconf einops

# Flash Attention 2 (선택사항, 성능 향상)
# !pip install flash-attn --no-build-isolation

print("의존성 설치 완료! (이미 설치된 경우 건너뛰기)")

In [ ]:
# Cell 5: 기본 import 및 환경 확인
"""
이 셀에서는 필요한 라이브러리를 import하고 환경을 확인합니다.

주요 라이브러리:
- torch: 딥러닝 프레임워크
- transformers: HuggingFace의 LLM 라이브러리
- matplotlib: 시각화
"""

import os
import sys
import torch
import torch.nn as nn
from typing import Optional, List, Tuple

# 시각화
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

# 환경 확인
print(f"Python 버전: {sys.version}")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA 버전: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Cell 6: 프로젝트 경로 설정 및 MemoryLLM import
"""
MemoryLLM 저장소의 코드를 import하기 위해 경로를 설정합니다.

파일 구조:
MemoryLLM/
├── modeling_memoryllm.py      # 추론용 모델 (이 파일을 import)
├── modeling_mplus.py          # M+ 확장 모델
├── configuration_memoryllm.py # 설정 클래스
├── test_qa_memory.py          # 평가 스크립트
└── train/                     # 학습 코드
"""

# 프로젝트 루트 경로 설정
PROJECT_ROOT = os.path.dirname(os.path.abspath('__file__'))  # 현재 노트북 위치
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print(f"프로젝트 루트: {PROJECT_ROOT}")

# MemoryLLM 관련 모듈 import
try:
    from modeling_memoryllm import MemoryLLM
    from configuration_memoryllm import MemoryLLMConfig
    print("✓ MemoryLLM 모듈 import 성공!")
except ImportError as e:
    print(f"✗ Import 실패: {e}")
    print("  → 이 노트북이 MemoryLLM 저장소 루트에 있는지 확인하세요.")

In [ ]:
# Cell 7: Tokenizer 로드
"""
LLaMA 기반 토크나이저를 로드합니다.

MemoryLLM은 LLaMA-2/3 기반이므로 해당 토크나이저를 사용합니다.
- LlamaTokenizer: 기본 LLaMA 토크나이저
- AutoTokenizer: HuggingFace의 자동 토크나이저 (권장)

주의: 실제 모델을 로드하려면 HuggingFace 로그인이 필요할 수 있습니다.
"""

from transformers import AutoTokenizer

# 모델 경로 (HuggingFace Hub 또는 로컬 경로)
MODEL_PATH = "YuWangX/memoryllm-8b"  # 또는 로컬 경로

# 토크나이저만 먼저 로드 (모델보다 가벼움)
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
    
    # 특수 토큰 확인
    print(f"Vocabulary 크기: {tokenizer.vocab_size}")
    print(f"PAD 토큰: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
    print(f"BOS 토큰: {tokenizer.bos_token} (ID: {tokenizer.bos_token_id})")
    print(f"EOS 토큰: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")
    print("✓ 토크나이저 로드 성공!")
except Exception as e:
    print(f"✗ 토크나이저 로드 실패: {e}")
    print("  → HuggingFace에 로그인하거나 로컬 모델 경로를 사용하세요.")
    tokenizer = None

In [ ]:
# Cell 8: 토큰화 예시
"""
토크나이저 사용 예시입니다.

MemoryLLM에서 토큰화는 두 가지 목적으로 사용됩니다:
1. Context 주입: inject_memory()에 전달할 context 토큰화
2. 질문/응답: 생성을 위한 입력 토큰화
"""

if tokenizer is not None:
    # 예시 텍스트
    context = "Last week, John had a wonderful picnic with David. During the picnic, David mentioned that his favorite fruit is strawberry."
    question = "What fruit does David like?"
    
    # 토큰화
    context_tokens = tokenizer(context, return_tensors='pt')
    question_tokens = tokenizer(question, return_tensors='pt')
    
    print("=== Context 토큰화 결과 ===")
    print(f"원문: {context}")
    print(f"토큰 수: {context_tokens['input_ids'].shape[1]}")
    print(f"토큰 ID (처음 10개): {context_tokens['input_ids'][0][:10].tolist()}")
    print()
    
    print("=== Question 토큰화 결과 ===")
    print(f"원문: {question}")
    print(f"토큰 수: {question_tokens['input_ids'].shape[1]}")
    print(f"토큰 ID: {question_tokens['input_ids'][0].tolist()}")
else:
    print("토크나이저가 로드되지 않았습니다.")

---

## Phase 1 완료!

이 섹션에서 다룬 내용:
- MemoryLLM 논문 소개 및 핵심 기여
- 아키텍처 개요
- 메모리 업데이트 메커니즘
- 환경 설정 및 기본 import
- 토크나이저 로드 및 사용 예시

다음 섹션에서는 **Core Architecture**를 상세히 분석합니다.

---

# Part 2: Core Architecture Deep Dive

이 섹션에서는 MemoryLLM의 핵심 아키텍처를 상세히 분석합니다.

## 2.1 Memory Pool 구조 상세 분석

Memory Pool은 MemoryLLM의 핵심 컴포넌트입니다. 각 차원의 의미를 정확히 이해하는 것이 중요합니다.

### Memory Pool Shape: `[L, num_blocks × num_tokens, hidden_size]`

```
Memory Pool 구조 시각화:

Layer 0:  [████████████████████████████████████████] ← num_blocks × num_tokens = 12800 tokens
Layer 1:  [████████████████████████████████████████]
Layer 2:  [████████████████████████████████████████]
  ...
Layer 31: [████████████████████████████████████████]

각 레이어: [num_blocks × num_tokens, hidden_size]
         = [50 × 256, 4096]
         = [12800, 4096]
```

### 차원별 의미

| 차원 | 변수명 | 기본값 | 의미 |
|------|--------|--------|------|
| **L** | `num_hidden_layers` | 32 | Transformer 레이어 수. 각 레이어마다 독립적인 메모리 슬라이스 보유 |
| **num_blocks** | `num_blocks` | 50 | 메모리 "창문" 개수. Drop 시 1/num_blocks 만큼 제거 |
| **num_tokens** | `num_tokens` | 256 | 한 번의 inject로 추가되는 토큰 수 |
| **hidden_size** | `hidden_size` | 4096 | 각 토큰의 표현 차원 (LLaMA-7B 기준) |

### 메모리 크기 계산

```
총 파라미터 = L × num_blocks × num_tokens × hidden_size
           = 32 × 50 × 256 × 4096
           = 1,677,721,600 ≈ 1.68B parameters
           ≈ 6.7 GB (float32) / 3.35 GB (float16)
```

In [ ]:
# Cell 10: Memory Pool 구조 시각화
"""
Memory Pool의 구조를 시각적으로 이해하기 위한 함수입니다.

이 시각화는 다음을 보여줍니다:
- 각 레이어별 메모리 슬라이스
- num_blocks 단위의 메모리 블록
- inject 시 새로 추가되는 영역
"""

def visualize_memory_structure(num_layers=32, num_blocks=50, num_tokens=256, 
                                highlight_new=True, show_layers=8):
    """
    Memory Pool 구조를 시각화합니다.
    
    Args:
        num_layers: Transformer 레이어 수
        num_blocks: 메모리 블록 수
        num_tokens: 블록당 토큰 수
        highlight_new: 새로 추가될 영역 강조
        show_layers: 표시할 레이어 수 (전체 표시하면 너무 김)
    """
    fig, ax = plt.subplots(figsize=(14, 6))
    
    total_tokens = num_blocks * num_tokens
    
    # 각 레이어별 메모리 바 그리기
    for i in range(show_layers):
        y = show_layers - 1 - i
        
        # 기존 메모리 (회색)
        ax.barh(y, total_tokens - num_tokens, left=0, height=0.8, 
                color='lightblue', edgecolor='navy', alpha=0.7)
        
        # 새로 추가될 메모리 (강조)
        if highlight_new:
            ax.barh(y, num_tokens, left=total_tokens - num_tokens, height=0.8,
                    color='coral', edgecolor='darkred', alpha=0.8)
        
        # 레이어 라벨
        ax.text(-500, y, f'Layer {i}', ha='right', va='center', fontsize=10)
    
    # ... 표시 (생략된 레이어)
    if num_layers > show_layers:
        ax.text(total_tokens / 2, -0.8, f'... (Layer {show_layers} ~ {num_layers-1})', 
                ha='center', fontsize=10, style='italic')
    
    # 블록 경계선 표시 (처음 5개 블록만)
    for b in range(6):
        x = b * num_tokens
        ax.axvline(x=x, color='gray', linestyle='--', alpha=0.3)
        if b < 5:
            ax.text(x + num_tokens/2, show_layers + 0.3, f'Block {b}', 
                    ha='center', fontsize=8, color='gray')
    
    # 범례
    existing_patch = mpatches.Patch(color='lightblue', label=f'기존 메모리 ({num_blocks-1} blocks)')
    new_patch = mpatches.Patch(color='coral', label=f'새 메모리 (1 block = {num_tokens} tokens)')
    ax.legend(handles=[existing_patch, new_patch], loc='upper right')
    
    # 축 설정
    ax.set_xlim(-1500, total_tokens + 500)
    ax.set_ylim(-1.5, show_layers + 1)
    ax.set_xlabel('Token Position')
    ax.set_title(f'Memory Pool Structure: [{num_layers}, {num_blocks}×{num_tokens}, hidden_size]')
    ax.set_yticks([])
    
    plt.tight_layout()
    plt.show()
    
    # 메모리 크기 정보 출력
    print(f"\n=== Memory Pool 정보 ===")
    print(f"Shape: [{num_layers}, {total_tokens}, 4096]")
    print(f"총 토큰 수: {num_layers * total_tokens:,}")
    print(f"파라미터 수: {num_layers * total_tokens * 4096:,} ({num_layers * total_tokens * 4096 / 1e9:.2f}B)")

# 시각화 실행
visualize_memory_structure()

## 2.2 MemoryLLMConfig 상세 분석

`MemoryLLMConfig`는 LLaMA의 `PretrainedConfig`를 상속받아 MemoryLLM 특화 파라미터를 추가합니다.

### 설정 파일 구조 (`configuration_memoryllm.py`)

```python
class MemoryLLMConfig(PretrainedConfig):
    model_type = "memoryllm"
    
    def __init__(
        self,
        # === LLaMA 기본 파라미터 ===
        vocab_size=32000,           # 어휘 크기
        hidden_size=4096,           # 은닉층 차원
        intermediate_size=11008,    # MLP 중간층 차원
        num_hidden_layers=32,       # Transformer 레이어 수
        num_attention_heads=32,     # 어텐션 헤드 수
        num_key_value_heads=None,   # GQA용 KV 헤드 수
        
        # === MemoryLLM 특화 파라미터 ===
        num_tokens=256,             # 한 번에 주입되는 토큰 수
        num_memory_tokens=12800,    # 총 메모리 토큰 수 (num_blocks × num_tokens)
        num_blocks=50,              # 메모리 블록 수
        add_bos_embedding=True,     # BOS 임베딩 추가 여부
        drop_memory_per_layer=False, # 레이어별 독립적 메모리 드롭
        add_decoder_lora=False,     # 디코더 LoRA 추가
        lora_config=None,           # LoRA 설정
        ...
    )
```

### MemoryLLM 특화 파라미터 설명

| 파라미터 | 기본값 | 설명 |
|---------|--------|------|
| `num_tokens` | 256 | `inject_memory()` 호출 시 추출되는 delta_memory의 토큰 수 |
| `num_memory_tokens` | 12800 | 전체 메모리 풀의 토큰 수 (`num_blocks × num_tokens`) |
| `num_blocks` | 50 | 메모리 슬라이딩 윈도우 크기. Drop 시 1/50 = 2% 제거 |
| `add_bos_embedding` | True | 각 레이어 메모리 앞에 BOS 임베딩 추가 |
| `drop_memory_per_layer` | False | True면 레이어마다 다른 인덱스 드롭 |

In [ ]:
# Cell 12: MemoryLLMConfig 생성 및 확인
"""
MemoryLLMConfig를 직접 생성하여 파라미터를 확인합니다.

이 설정은 모델 초기화 시 다음을 결정합니다:
- Memory Pool의 크기와 구조
- 어텐션 메커니즘 설정
- 메모리 드롭 전략
"""

# 기본 설정으로 Config 생성
config = MemoryLLMConfig(
    # LLaMA 기본 설정
    vocab_size=32000,
    hidden_size=4096,
    intermediate_size=11008,
    num_hidden_layers=32,
    num_attention_heads=32,
    
    # MemoryLLM 특화 설정
    num_tokens=256,         # 한 번에 주입되는 토큰 수
    num_blocks=50,          # 메모리 블록 수
    num_memory_tokens=256 * 50,  # 자동 계산: 12800
    add_bos_embedding=True,
    drop_memory_per_layer=False,
)

print("=== MemoryLLMConfig 설정 확인 ===\n")

# LLaMA 기본 설정
print("[LLaMA 기본 설정]")
print(f"  vocab_size: {config.vocab_size}")
print(f"  hidden_size: {config.hidden_size}")
print(f"  intermediate_size: {config.intermediate_size}")
print(f"  num_hidden_layers: {config.num_hidden_layers}")
print(f"  num_attention_heads: {config.num_attention_heads}")
print()

# MemoryLLM 특화 설정
print("[MemoryLLM 특화 설정]")
print(f"  num_tokens: {config.num_tokens}")
print(f"  num_blocks: {config.num_blocks}")
print(f"  num_memory_tokens: {config.num_memory_tokens}")
print(f"  add_bos_embedding: {config.add_bos_embedding}")
print(f"  drop_memory_per_layer: {config.drop_memory_per_layer}")
print()

# 계산된 값
print("[계산된 메모리 정보]")
memory_params = config.num_hidden_layers * config.num_memory_tokens * config.hidden_size
print(f"  Memory Pool Shape: [{config.num_hidden_layers}, {config.num_memory_tokens}, {config.hidden_size}]")
print(f"  Memory Parameters: {memory_params:,} ({memory_params / 1e9:.2f}B)")
print(f"  Drop 비율: 1/{config.num_blocks} = {1/config.num_blocks*100:.1f}%")
print(f"  Drop 토큰 수: {config.num_memory_tokens // config.num_blocks} tokens")

## 2.3 MemoryLLM 클래스 구조

`MemoryLLM`은 HuggingFace의 `LlamaForCausalLM`을 상속받아 메모리 기능을 추가합니다.

### 클래스 계층 구조

```
MemoryLLM (modeling_memoryllm.py)
    └── LlamaForCausalLM (transformers)
            └── LlamaPreTrainedModel
                    └── PreTrainedModel
```

### `__init__` 메서드 분석 (Line 1512-1550)

```python
class MemoryLLM(LlamaForCausalLM):
    def __init__(self, config):
        # 1. LLaMA 기본 초기화
        LlamaForCausalLM.__init__(self, config)
        
        # 2. Config에서 핵심 파라미터 추출
        self.L = config.num_hidden_layers      # 레이어 수 (32)
        self.d = config.hidden_size            # 은닉층 차원 (4096)
        self.num_blocks = config.num_blocks    # 메모리 블록 수 (50)
        self.num_tokens = config.num_tokens    # 블록당 토큰 수 (256)
        
        # 3. Memory Pool 초기화 (핵심!)
        self.memory = nn.Parameter(torch.randn([self.L, self.num_blocks * self.num_tokens, self.d]))
        self.memory.requires_grad = False  # 추론 시 학습 비활성화
        
        # 4. 초기화 상태 추적 (Buffer로 저장)
        self.register_buffer("initialized", torch.tensor(0, dtype=torch.uint8))
        
        # 5. 위치 임베딩
        self.new_memory_positional_emb = nn.Parameter(torch.zeros([1, 1, self.d]))
        
        # 6. BOS 임베딩 (선택)
        if config.add_bos_embedding:
            self.bos_embedding = nn.Parameter(torch.randn([self.L, 1, self.d]))
```

### 핵심 속성 설명

| 속성 | Shape | 설명 |
|------|-------|------|
| `memory` | `[L, num_blocks×num_tokens, d]` | 메인 메모리 풀. 컨텍스트 정보 저장 |
| `initialized` | `scalar (uint8)` | 메모리 초기화 여부 (0: 미초기화, 1+: 초기화됨) |
| `new_memory_positional_emb` | `[1, 1, d]` | 새 메모리에 추가되는 위치 임베딩 |
| `bos_embedding` | `[L, 1, d]` | 각 레이어의 시작 토큰 임베딩 |

In [ ]:
# Cell 14: MemoryLLM 모델 로드 (선택적)
"""
실제 MemoryLLM 모델을 로드합니다.

주의: 이 셀은 GPU 메모리를 많이 사용합니다 (~18GB for 8B model).
실행하기 전에 충분한 GPU 메모리가 있는지 확인하세요.

로드 옵션:
- torch_dtype=torch.float16: 메모리 절약
- attn_implementation="flash_attention_2": 속도 향상 (선택)
- device_map="auto": 자동 디바이스 배치
"""

# GPU 메모리 확인
if torch.cuda.is_available():
    free_memory = torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0)
    print(f"사용 가능한 GPU 메모리: {free_memory / 1e9:.1f} GB")
    
    if free_memory < 16e9:
        print("⚠️  경고: GPU 메모리가 16GB 미만입니다. 모델 로딩이 실패할 수 있습니다.")
        LOAD_MODEL = False
    else:
        LOAD_MODEL = True
else:
    print("⚠️  CUDA를 사용할 수 없습니다. CPU로 실행하면 매우 느립니다.")
    LOAD_MODEL = False

# 모델 로드 (선택적)
model = None
if LOAD_MODEL:
    print("\n모델을 로드합니다... (약 1-2분 소요)")
    
    try:
        model = MemoryLLM.from_pretrained(
            MODEL_PATH,
            torch_dtype=torch.float16,  # 메모리 절약
            # attn_implementation="flash_attention_2",  # Flash Attention (선택)
            device_map="auto",  # 자동 디바이스 배치
            trust_remote_code=True,
        )
        print("✓ 모델 로드 완료!")
        
        # 모델 정보 출력
        print(f"\n=== 모델 정보 ===")
        print(f"모델 타입: {type(model).__name__}")
        print(f"num_hidden_layers (L): {model.L}")
        print(f"hidden_size (d): {model.d}")
        print(f"num_blocks: {model.num_blocks}")
        print(f"num_tokens: {model.num_tokens}")
        print(f"Memory Pool Shape: {model.memory.shape}")
        print(f"Initialized: {model.initialized.item()}")
        
    except Exception as e:
        print(f"✗ 모델 로드 실패: {e}")
        model = None
else:
    print("\n모델 로딩을 건너뜁니다. (GPU 메모리 부족 또는 CUDA 미사용)")

## 2.4 MemoryLLM 핵심 메서드 개요

MemoryLLM의 주요 메서드들을 요약합니다. 각 메서드의 상세 분석은 다음 섹션에서 다룹니다.

### 메서드 요약표

| 메서드 | 위치 | 역할 |
|--------|------|------|
| `inject_memory()` | Line 1553 | 컨텍스트를 메모리에 주입 |
| `drop_memory()` | Line 1573 | 메모리 풀에서 일부 제거 |
| `update_memory_with_delta_memory()` | Line 1603 | delta_memory로 메모리 풀 갱신 |
| `cat_memory_and_hiddens()` | Line 1673 | 메모리와 hidden states 연결 |
| `forward()` | Line 1747 | 전체 forward pass |

### 메서드 호출 흐름

```
사용자 코드:
    model.inject_memory(context_ids, update_memory=True)
    output = model.generate(question_ids)

내부 호출 흐름:
    inject_memory()
        └── forward(is_injection=True, output_delta_memory=True)
                └── cat_memory_and_hiddens() (각 레이어에서)
                └── delta_memory 추출
        └── update_memory_with_delta_memory()
                └── drop_memory()
                └── torch.cat([old_memory, delta_memory])
    
    generate()
        └── forward(is_injection=False)
                └── cat_memory_and_hiddens() (메모리 포함)
                └── 토큰 생성
```

### 중요 플래그

| 플래그 | 기본값 | 설명 |
|--------|--------|------|
| `is_injection` | False | True: 메모리 주입 모드, False: 생성 모드 |
| `output_delta_memory` | False | True: delta_memory 반환 |
| `update_memory` | False | True: 메모리 풀 실제 갱신 |

---

## Phase 2 완료!

이 섹션에서 다룬 내용:
- Memory Pool 구조 상세 분석 (`[L, num_blocks × num_tokens, hidden_size]`)
- MemoryLLMConfig 파라미터 이해
- MemoryLLM 클래스 초기화 과정
- 핵심 메서드 개요

다음 섹션에서는 **Memory Operations** (inject_memory, update_memory 등)를 상세히 분석합니다.

---

# Part 3: Memory Operations - Injection & Update

이 섹션에서는 MemoryLLM의 핵심 메모리 연산을 상세히 분석합니다.

## 3.1 inject_memory() 메서드 분석

`inject_memory()`는 새로운 컨텍스트를 메모리에 주입하는 핵심 메서드입니다.

### 메서드 시그니처 (Line 1553-1570)

```python
def inject_memory(self, context_ids, 
                  context_attention_mask=None,
                  delta_memory=None,
                  update_memory=False):
    """
    컨텍스트를 메모리에 주입합니다.
    
    Args:
        context_ids: 주입할 컨텍스트의 토큰 ID [batch, seq_len]
        context_attention_mask: 어텐션 마스크 [batch, seq_len]
        delta_memory: 기존에 계산된 delta_memory (다중 컨텍스트 처리용)
        update_memory: True면 메모리 풀을 실제로 갱신
    
    Returns:
        delta_memory: 추출된 메모리 표현 [batch, L, num_tokens, d]
    """
    # Forward pass with injection mode
    output = self(
        input_ids=context_ids,
        attention_mask=context_attention_mask,
        delta_memory=delta_memory,
        is_injection=True,           # 주입 모드 활성화
        output_delta_memory=True,    # delta_memory 출력
        return_dict=True
    )
    
    # 메모리 업데이트 (선택)
    if update_memory:
        self.update_memory_with_delta_memory(output.delta_memory)
    
    return output.delta_memory
```

### 동작 과정

```
입력: context_ids [batch, seq_len]
      예: "Last week, John had a picnic with David..." (30 tokens)

      ↓ forward(is_injection=True)

각 레이어에서:
    1. hidden_states 계산
    2. 마지막 num_tokens (256) 추출 → delta_memory[layer_idx]

      ↓ stack across layers

출력: delta_memory [batch, L, num_tokens, d]
      예: [1, 32, 256, 4096]
```

In [ ]:
# Cell 18: inject_memory 사용 예시 (시뮬레이션)
"""
inject_memory()의 동작을 시뮬레이션합니다.

실제 모델 없이도 메모리 주입 과정을 이해할 수 있도록
간단한 예시를 제공합니다.
"""

def simulate_inject_memory(context_length, num_layers=32, num_tokens=256, hidden_size=4096):
    """
    inject_memory 동작을 시뮬레이션합니다.
    
    Args:
        context_length: 입력 컨텍스트의 토큰 수
        num_layers: Transformer 레이어 수
        num_tokens: 추출할 토큰 수
        hidden_size: 은닉층 차원
    
    Returns:
        delta_memory shape 정보
    """
    print(f"=== inject_memory 시뮬레이션 ===\n")
    
    # 입력
    print(f"[입력]")
    print(f"  context_length: {context_length} tokens")
    print(f"  → 예: 'Last week, John had a picnic...' ({context_length} tokens)")
    print()
    
    # Forward pass
    print(f"[Forward Pass]")
    print(f"  is_injection=True")
    print(f"  output_delta_memory=True")
    print()
    
    # 각 레이어에서 delta_memory 추출
    print(f"[각 레이어에서 delta_memory 추출]")
    for i in [0, 1, 2, "...", num_layers-1]:
        if i == "...":
            print(f"  ...")
        else:
            print(f"  Layer {i}: hidden_states[:, -{num_tokens}:, :] → [{1}, {num_tokens}, {hidden_size}]")
    print()
    
    # 최종 출력
    print(f"[출력: delta_memory]")
    print(f"  Shape: [batch=1, L={num_layers}, num_tokens={num_tokens}, d={hidden_size}]")
    print(f"  메모리 크기: {1 * num_layers * num_tokens * hidden_size * 2 / 1e6:.1f} MB (float16)")
    print()
    
    # 핵심 인사이트
    print(f"[핵심 인사이트]")
    if context_length < num_tokens:
        print(f"  ⚠️  context_length ({context_length}) < num_tokens ({num_tokens})")
        print(f"     → 컨텍스트가 num_tokens보다 작으면 패딩 또는 반복됨")
    else:
        print(f"  ✓ context_length ({context_length}) >= num_tokens ({num_tokens})")
        print(f"     → 마지막 {num_tokens} 토큰의 hidden states가 delta_memory가 됨")

# 시뮬레이션 실행
simulate_inject_memory(context_length=30)
print("\n" + "="*50 + "\n")
simulate_inject_memory(context_length=512)

## 3.2 update_memory_with_delta_memory() 분석

`update_memory_with_delta_memory()`는 추출된 delta_memory로 메모리 풀을 갱신합니다.

### 메서드 로직 (Line 1603-1671)

```python
def update_memory_with_delta_memory(self, delta_memory):
    # Shape 정리: [batch, L, num_tokens, d] → [L, num_tokens, d]
    if len(delta_memory.shape) == 4:
        delta_memory = delta_memory.detach()[0]
    
    # === 경우 1: 첫 번째 초기화 (initialized == 0) ===
    if self.initialized == 0:
        # delta_memory가 메모리 풀 크기보다 작으면 반복해서 채움
        if delta_memory.shape[1] < (self.num_tokens * self.num_blocks):
            delta_memory = torch.cat([delta_memory] * k, dim=1)  # k번 반복
        else:
            # 또는 마지막 부분만 사용
            delta_memory = delta_memory[:, -self.num_tokens * self.num_blocks:]
        
        self.memory.data = delta_memory  # 전체 교체
    
    # === 경우 2: 이후 업데이트 (initialized > 0) ===
    else:
        # 기존 메모리에서 일부 드롭
        current_memory = self.drop_memory(self.memory.data.detach())
        
        # 새 메모리 추가
        self.memory.data = torch.cat([current_memory, delta_memory], dim=1)
    
    # 초기화 플래그 증가
    if not self.initialized:
        self.initialized += 1
```

### 두 가지 경우 시각화

```
=== 경우 1: 첫 번째 초기화 (initialized == 0) ===

delta_memory: [████████]  (num_tokens = 256)
                   ↓ 반복
Memory Pool:  [████████████████████████████████████████]  (12800 tokens)
              ^--- delta_memory로 전체 채움 ---^


=== 경우 2: 이후 업데이트 (initialized > 0) ===

기존 메모리:  [████████████████████████████████████████]  (12800 tokens)
                   ↓ drop_memory() (1/50 = 256 tokens 제거)
드롭 후:      [██████████████████████████████████████░░]  (12544 tokens)
                   ↓ + delta_memory
새 메모리:    [██████████████████████████████████████████]  (12800 tokens)
                                                ^^^^
                                            새로 추가된 부분
```

## 3.3 drop_memory() 분석

`drop_memory()`는 메모리 풀에서 공간을 확보하기 위해 일부를 제거합니다.

### 메서드 로직 (Line 1573-1601)

```python
def drop_memory(self, current_memory, drop_length=None, unsequeezed=True):
    """
    메모리 풀에서 일부를 랜덤하게 제거합니다.
    
    Args:
        current_memory: 현재 메모리 [L, total_tokens, d]
        drop_length: 제거할 토큰 수 (기본: 1/num_blocks)
        unsequeezed: True면 [L, tokens, d], False면 [tokens, d]
    
    Returns:
        드롭된 메모리 [L, total_tokens - drop_length, d]
    """
    if unsequeezed:
        # 드롭할 길이 계산 (기본: 1/num_blocks = 2%)
        if drop_length is None:
            drop_length = int(current_memory.shape[1] * (1 / self.num_blocks))
        
        # 남길 인덱스 랜덤 선택
        left_indices = torch.randperm(current_memory.shape[1])[
            :current_memory.shape[1] - drop_length
        ]
        
        # 인덱스 정렬 (순서 유지)
        left_indices = left_indices.sort()[0]
        
        # 선택된 인덱스만 유지
        current_memory = current_memory[:, left_indices, :]
        
        return current_memory
```

### 핵심 특징

1. **랜덤 드롭**: 가장 오래된 메모리가 아닌 **랜덤하게** 제거
   - 이유: 오래된 정보도 여전히 중요할 수 있음
   - 효과: 메모리의 다양성 유지

2. **인덱스 정렬**: 드롭 후에도 남은 인덱스는 오름차순 유지
   - 이유: 상대적 순서 보존
   - 효과: 위치 정보의 일관성

3. **레이어 공유 드롭** (기본): 모든 레이어에서 동일한 인덱스 드롭
   - `drop_memory_per_layer=False` (기본)
   - `drop_memory_per_layer=True`: 레이어별 독립적 드롭

In [ ]:
# Cell 21: drop_memory 동작 시각화
"""
drop_memory()의 랜덤 드롭 동작을 시각화합니다.

핵심 포인트:
- 랜덤하게 선택 → 정렬 → 순서 유지
- 새 메모리는 항상 끝에 추가
"""

def visualize_drop_memory(num_blocks=50, num_tokens=256):
    """drop_memory 동작을 시각화합니다."""
    
    total_tokens = num_blocks * num_tokens
    drop_length = total_tokens // num_blocks  # 1/num_blocks
    
    # 시뮬레이션: 랜덤 인덱스 선택
    np.random.seed(42)  # 재현성을 위해
    all_indices = np.arange(total_tokens)
    
    # 드롭할 인덱스 (랜덤)
    drop_indices = np.random.choice(all_indices, drop_length, replace=False)
    drop_indices.sort()
    
    # 남길 인덱스
    keep_indices = np.setdiff1d(all_indices, drop_indices)
    
    # 시각화
    fig, axes = plt.subplots(3, 1, figsize=(14, 8))
    
    # 1. 원본 메모리
    ax1 = axes[0]
    ax1.barh(0, total_tokens, height=0.6, color='lightblue', edgecolor='navy')
    ax1.set_xlim(0, total_tokens + 500)
    ax1.set_ylim(-0.5, 0.5)
    ax1.set_yticks([])
    ax1.set_title(f'1. 원본 메모리: {total_tokens:,} tokens ({num_blocks} blocks × {num_tokens} tokens)')
    ax1.axvline(x=total_tokens - num_tokens, color='red', linestyle='--', label='New memory position')
    
    # 2. 드롭된 위치 표시
    ax2 = axes[1]
    ax2.barh(0, total_tokens, height=0.6, color='lightblue', edgecolor='navy', alpha=0.5)
    # 드롭 위치 표시 (샘플링하여 표시)
    sample_drops = drop_indices[::10]  # 10개마다 표시
    for idx in sample_drops:
        ax2.axvline(x=idx, color='red', alpha=0.3, linewidth=1)
    ax2.set_xlim(0, total_tokens + 500)
    ax2.set_ylim(-0.5, 0.5)
    ax2.set_yticks([])
    ax2.set_title(f'2. 랜덤 드롭: {drop_length} tokens 제거 (빨간 선 = 드롭 위치, 샘플)')
    
    # 3. 드롭 후 + 새 메모리
    ax3 = axes[2]
    remaining = total_tokens - drop_length
    ax3.barh(0, remaining, height=0.6, color='lightblue', edgecolor='navy', label=f'기존 메모리 ({remaining:,})')
    ax3.barh(0, num_tokens, left=remaining, height=0.6, color='coral', edgecolor='darkred', label=f'새 메모리 ({num_tokens})')
    ax3.set_xlim(0, total_tokens + 500)
    ax3.set_ylim(-0.5, 0.5)
    ax3.set_yticks([])
    ax3.set_title(f'3. 드롭 후 + 새 메모리: {remaining:,} + {num_tokens} = {remaining + num_tokens:,} tokens')
    ax3.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()
    
    # 통계 출력
    print(f"\n=== drop_memory 통계 ===")
    print(f"원본 메모리: {total_tokens:,} tokens")
    print(f"드롭 비율: 1/{num_blocks} = {100/num_blocks:.1f}%")
    print(f"드롭 토큰 수: {drop_length:,} tokens")
    print(f"남은 토큰 수: {remaining:,} tokens")
    print(f"새 메모리 추가: {num_tokens} tokens")
    print(f"최종 메모리: {remaining + num_tokens:,} tokens (원본과 동일)")

visualize_drop_memory()

## 3.4 cat_memory_and_hiddens() 분석

`cat_memory_and_hiddens()`는 각 레이어에서 메모리와 현재 hidden states를 연결합니다.

### 메서드 로직 (Line 1673-1745)

```python
def cat_memory_and_hiddens(self, idx, hidden_states, delta_memory=None, 
                           is_injection=False, cat_to_maximum_memory=False):
    """
    메모리와 hidden states를 연결합니다.
    
    Args:
        idx: 현재 레이어 인덱스
        hidden_states: 현재 레이어의 입력 [batch, seq_len, d]
        delta_memory: 이전에 계산된 delta_memory (선택)
        is_injection: 주입 모드 여부
    
    Returns:
        연결된 hidden states [batch, memory_len + seq_len, d]
    """
    # 메모리가 초기화되지 않았으면 그냥 반환
    if not self.initialized:
        return hidden_states
    
    # === 경우 1: delta_memory가 없는 경우 ===
    if delta_memory is None or len(delta_memory) == 0:
        if is_injection:
            # 주입 모드: 마지막 num_tokens만 사용
            cur_memory = self.memory[idx][-self.num_tokens:]  # [num_tokens, d]
        else:
            # 생성 모드: 전체 메모리 사용
            cur_memory = self.memory[idx]  # [total_tokens, d]
    
    # === 경우 2: delta_memory가 있는 경우 ===
    else:
        cur_memory = delta_memory[:, idx]  # [batch, num_tokens, d]
        
        if not is_injection:
            # 생성 모드에서는 old_memory도 샘플링하여 연결
            old_memory = self.memory[idx].detach()
            sampled_indices = torch.randperm(old_memory.shape[0])[
                :old_memory.shape[0] - cur_memory.shape[1]
            ].sort()[0]
            old_memory = old_memory[sampled_indices]
            cur_memory = torch.cat([old_memory, cur_memory], dim=1)
    
    # BOS 임베딩 추가 (선택)
    if self.add_bos_embedding:
        cur_memory = torch.cat([self.bos_embedding[idx], cur_memory], dim=1)
    
    # 메모리 + hidden_states 연결
    return torch.cat([cur_memory, hidden_states], dim=1)
```

### 동작 시각화

```
=== is_injection=True (주입 모드) ===

cur_memory:    [████]              (최근 num_tokens = 256)
hidden_states:        [██████████] (입력 seq_len = 30)
                   ↓ cat
결과:          [████][██████████]  (256 + 30 = 286)


=== is_injection=False (생성 모드) ===

cur_memory:    [████████████████████████████████████████] (전체 12800)
hidden_states:                                           [██] (질문 10)
                   ↓ cat
결과:          [████████████████████████████████████████][██] (12800 + 10 = 12810)
```

In [ ]:
# Cell 23: 메모리 연산 종합 예제 (실제 모델 사용 시)
"""
실제 모델이 로드된 경우 메모리 연산을 수행합니다.

이 셀은 다음을 시연합니다:
1. inject_memory()로 컨텍스트 주입
2. 메모리 상태 확인
3. 추가 컨텍스트 주입
4. 메모리 변화 관찰
"""

def demonstrate_memory_operations(model, tokenizer):
    """메모리 연산을 시연합니다."""
    
    print("=== 메모리 연산 시연 ===\n")
    
    # 1. 초기 상태 확인
    print("[1. 초기 상태]")
    print(f"  initialized: {model.initialized.item()}")
    print(f"  memory shape: {model.memory.shape}")
    print()
    
    # 2. 첫 번째 컨텍스트 주입
    context1 = "John's favorite color is blue. He always wears blue shirts and has a blue car."
    print(f"[2. 첫 번째 컨텍스트 주입]")
    print(f"  컨텍스트: '{context1[:50]}...'")
    
    context1_ids = tokenizer(context1, return_tensors='pt').input_ids.to(model.device)
    print(f"  토큰 수: {context1_ids.shape[1]}")
    
    # 메모리 백업
    memory_before = model.memory.data.clone()
    
    # 주입
    delta_memory1 = model.inject_memory(context1_ids, update_memory=True)
    
    print(f"  delta_memory shape: {delta_memory1.shape}")
    print(f"  initialized after: {model.initialized.item()}")
    
    # 메모리 변화 확인
    memory_after = model.memory.data
    memory_diff = (memory_after - memory_before).abs().mean().item()
    print(f"  메모리 변화량 (mean abs diff): {memory_diff:.6f}")
    print()
    
    # 3. 두 번째 컨텍스트 주입
    context2 = "Mary loves strawberries. She eats strawberries every morning for breakfast."
    print(f"[3. 두 번째 컨텍스트 주입]")
    print(f"  컨텍스트: '{context2[:50]}...'")
    
    context2_ids = tokenizer(context2, return_tensors='pt').input_ids.to(model.device)
    print(f"  토큰 수: {context2_ids.shape[1]}")
    
    memory_before2 = model.memory.data.clone()
    
    delta_memory2 = model.inject_memory(context2_ids, update_memory=True)
    
    memory_after2 = model.memory.data
    memory_diff2 = (memory_after2 - memory_before2).abs().mean().item()
    print(f"  메모리 변화량: {memory_diff2:.6f}")
    print()
    
    # 4. 최종 상태
    print(f"[4. 최종 상태]")
    print(f"  총 주입 횟수: 2")
    print(f"  메모리 shape: {model.memory.shape}")
    print(f"  각 주입으로 인한 메모리 갱신:")
    print(f"    - drop: 1/{model.num_blocks} = {model.num_tokens} tokens")
    print(f"    - add: {model.num_tokens} tokens (delta_memory)")
    
    return delta_memory1, delta_memory2

# 실행 (모델이 로드된 경우에만)
if model is not None and tokenizer is not None:
    delta1, delta2 = demonstrate_memory_operations(model, tokenizer)
else:
    print("모델이 로드되지 않았습니다.")
    print("위의 '모델 로드' 셀을 먼저 실행하거나,")
    print("코드를 참고하여 메모리 연산 흐름을 이해하세요.")

---

## Phase 3 완료!

이 섹션에서 다룬 내용:
- `inject_memory()`: 컨텍스트를 메모리에 주입
- `update_memory_with_delta_memory()`: delta_memory로 메모리 풀 갱신
- `drop_memory()`: 랜덤 드롭으로 공간 확보
- `cat_memory_and_hiddens()`: 메모리와 hidden states 연결

### 핵심 정리

| 메서드 | 역할 | 입력 | 출력 |
|--------|------|------|------|
| `inject_memory` | 컨텍스트 주입 | context_ids | delta_memory |
| `update_memory_with_delta_memory` | 메모리 갱신 | delta_memory | None (in-place) |
| `drop_memory` | 공간 확보 | current_memory | dropped_memory |
| `cat_memory_and_hiddens` | 연결 | hidden_states | [memory, hidden_states] |

다음 섹션에서는 **Forward Pass & Generation**을 상세히 분석합니다.

---

# Part 4: Forward Pass & Generation

이 섹션에서는 MemoryLLM의 forward pass와 텍스트 생성을 상세히 분석합니다.

## 4.1 forward() 메서드 시그니처

`forward()` 메서드는 MemoryLLM의 핵심 연산을 수행합니다.

### 메서드 시그니처 (Line 1747-1764)

```python
def forward(
    self,
    input_ids: torch.LongTensor = None,           # 입력 토큰 ID
    attention_mask: Optional[torch.Tensor] = None, # 어텐션 마스크
    position_ids: Optional[torch.LongTensor] = None,
    past_key_values: Optional[List[torch.FloatTensor]] = None,  # KV 캐시
    inputs_embeds: Optional[torch.FloatTensor] = None,
    delta_memory: Optional[List[List[torch.FloatTensor]]] = None,  # 메모리 (MemoryLLM 추가)
    labels: torch.LongTensor = None,              # 학습용 레이블
    use_cache: Optional[bool] = None,
    output_attentions: Optional[bool] = None,
    output_delta_memory: Optional[bool] = None,   # delta_memory 출력 (MemoryLLM 추가)
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None,
    cache_position: Optional[torch.LongTensor] = None,
    is_injection: Optional[bool] = None,          # 주입 모드 (MemoryLLM 추가)
    cat_to_maximum_memory: Optional[bool] = False,
) -> Union[Tuple, MemoryLMOutputWithPastAndCrossAttentions]:
```

### MemoryLLM 추가 파라미터

| 파라미터 | 타입 | 설명 |
|---------|------|------|
| `delta_memory` | `[batch, L, num_tokens, d]` | 이전에 계산된 메모리 (다중 컨텍스트 처리) |
| `output_delta_memory` | `bool` | True면 delta_memory 반환 |
| `is_injection` | `bool` | True: 주입 모드, False: 생성 모드 |
| `cat_to_maximum_memory` | `bool` | 최대 메모리 크기로 연결 |

## 4.2 Forward Pass 단계별 분석

### 전체 흐름

```
입력: input_ids [batch, seq_len]
       ↓
1. Embedding: inputs_embeds [batch, seq_len, d]
       ↓
2. Cache Position 계산 (메모리 고려)
       ↓
3. 각 Decoder Layer 순회:
   for idx in range(num_layers):
       a. cat_memory_and_hiddens() → [batch, memory_len + seq_len, d]
       b. new_memory_positional_emb 추가 (is_injection일 때)
       c. decoder_layer forward
       d. delta_memory 추출 (output_delta_memory일 때)
       e. 원래 seq_len으로 복원
       ↓
4. Final Layer Norm
       ↓
5. LM Head → logits [batch, seq_len, vocab_size]
       ↓
6. Loss 계산 (labels가 있으면)
       ↓
출력: MemoryLMOutputWithPastAndCrossAttentions
      - loss, logits, delta_memory, past_key_values, ...
```

### 핵심 로직: Cache Position 계산

```python
# 메모리가 초기화된 경우
if self.initialized:
    if is_injection:
        # 주입 모드: 최근 메모리 + 입력
        cache_position = torch.arange(
            0, inputs_embeds.shape[1] + self.num_tokens + int(self.add_bos_embedding)
        )
        # 예: [0, 1, 2, ..., 256 + 30 + 1] = 287 positions
    else:
        # 생성 모드: 전체 메모리 + 입력
        cache_position = torch.arange(
            0, inputs_embeds.shape[1] + self.num_tokens * self.num_blocks + int(self.add_bos_embedding)
        )
        # 예: [0, 1, 2, ..., 12800 + 10 + 1] = 12811 positions
```

### RoPE (Rotary Position Embedding) 처리

MemoryLLM은 긴 position을 처리하기 위해 RoPE scaling을 사용합니다:

```python
# config에서 설정
if rope_scaling is not None:
    rope_scaling['factor'] = (num_memory_tokens + max_length) / max_position_embeddings
    # 예: (12800 + 2048) / 4096 ≈ 3.6x scaling
```

In [ ]:
# Cell 27: Forward Pass 시뮬레이션
"""
Forward Pass의 각 단계를 시뮬레이션합니다.

실제 연산 없이 shape 변화를 추적하여 흐름을 이해합니다.
"""

def simulate_forward_pass(seq_len, num_layers=32, num_blocks=50, num_tokens=256, 
                          hidden_size=4096, is_injection=True, initialized=True):
    """
    Forward pass를 시뮬레이션합니다.
    
    Args:
        seq_len: 입력 시퀀스 길이
        num_layers: 레이어 수
        num_blocks: 메모리 블록 수
        num_tokens: 블록당 토큰 수
        hidden_size: 은닉층 차원
        is_injection: 주입 모드 여부
        initialized: 메모리 초기화 여부
    """
    mode = "주입 모드 (is_injection=True)" if is_injection else "생성 모드 (is_injection=False)"
    print(f"=== Forward Pass 시뮬레이션: {mode} ===\n")
    
    # 1. 입력
    print(f"[1. 입력]")
    print(f"  input_ids shape: [1, {seq_len}]")
    print()
    
    # 2. Embedding
    print(f"[2. Embedding]")
    print(f"  inputs_embeds shape: [1, {seq_len}, {hidden_size}]")
    print()
    
    # 3. Cache Position 계산
    print(f"[3. Cache Position 계산]")
    if not initialized:
        cache_len = seq_len
        print(f"  (메모리 미초기화)")
    elif is_injection:
        # 주입 모드: 최근 메모리 + 입력 + BOS
        memory_len = num_tokens
        cache_len = memory_len + seq_len + 1  # +1 for BOS
        print(f"  memory_len (recent): {memory_len}")
    else:
        # 생성 모드: 전체 메모리 + 입력 + BOS
        memory_len = num_blocks * num_tokens
        cache_len = memory_len + seq_len + 1  # +1 for BOS
        print(f"  memory_len (full): {memory_len}")
    
    print(f"  cache_position: [0, 1, ..., {cache_len-1}] (length: {cache_len})")
    print()
    
    # 4. Decoder Layers
    print(f"[4. Decoder Layers]")
    for layer_idx in [0, 1, "...", num_layers-1]:
        if layer_idx == "...":
            print(f"  ...")
            continue
            
        print(f"  Layer {layer_idx}:")
        
        # cat_memory_and_hiddens
        if initialized:
            if is_injection:
                concat_len = num_tokens + seq_len
            else:
                concat_len = num_blocks * num_tokens + seq_len
            print(f"    a. cat_memory_and_hiddens → [1, {concat_len}, {hidden_size}]")
            
            if is_injection:
                print(f"    b. + new_memory_positional_emb (마지막 {num_tokens} tokens)")
        else:
            concat_len = seq_len
            print(f"    a. 메모리 없음 → [1, {seq_len}, {hidden_size}]")
        
        # decoder_layer forward
        print(f"    c. decoder_layer forward")
        
        # delta_memory 추출
        if is_injection:
            print(f"    d. delta_memory[{layer_idx}] = hidden_states[:, -{num_tokens}:, :]")
        
        # 복원
        print(f"    e. hidden_states = hidden_states[:, -{seq_len}:, :] → [1, {seq_len}, {hidden_size}]")
        print()
    
    # 5. LM Head
    vocab_size = 32000
    print(f"[5. LM Head]")
    print(f"  logits shape: [1, {seq_len}, {vocab_size}]")
    print()
    
    # 6. 출력
    print(f"[6. 출력]")
    print(f"  logits: [1, {seq_len}, {vocab_size}]")
    if is_injection:
        print(f"  delta_memory: [1, {num_layers}, {num_tokens}, {hidden_size}]")

# 주입 모드 시뮬레이션
simulate_forward_pass(seq_len=30, is_injection=True)

print("\n" + "="*60 + "\n")

# 생성 모드 시뮬레이션
simulate_forward_pass(seq_len=10, is_injection=False)

## 4.3 텍스트 생성 (Generation)

MemoryLLM에서 텍스트 생성은 HuggingFace의 `generate()` 메서드를 그대로 사용합니다.

### 기본 사용법

```python
# 1. 컨텍스트 주입
context = "John's favorite fruit is apple. He eats apples every day."
context_ids = tokenizer(context, return_tensors='pt').input_ids.cuda()
model.inject_memory(context_ids, update_memory=True)

# 2. 질문 생성
question = "What is John's favorite fruit?"
question_ids = tokenizer(question, return_tensors='pt').input_ids.cuda()

# 3. 응답 생성
output = model.generate(
    inputs=question_ids,
    max_new_tokens=20,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

# 4. 디코딩
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)  # "What is John's favorite fruit? Apple."
```

### 생성 파라미터

| 파라미터 | 설명 | 권장값 |
|---------|------|--------|
| `max_new_tokens` | 생성할 최대 토큰 수 | 10-50 (QA), 100-500 (긴 응답) |
| `temperature` | 샘플링 온도 | 0.7-1.0 |
| `top_p` | Nucleus sampling | 0.9-0.95 |
| `do_sample` | 샘플링 사용 여부 | True (다양성), False (결정적) |
| `num_beams` | Beam search | 1-5 |

### 생성 시 메모리 활용

```
생성 모드에서 각 레이어:
    hidden_states = cat_memory_and_hiddens(idx, hidden_states)
    
    결과: [전체 메모리 (12800 tokens)] + [질문 (10 tokens)]
         ^^^^^^^^^^^^^^^^^^^^^^^^^
         이 메모리 덕분에 주입된 컨텍스트 정보 활용 가능
```

In [ ]:
# Cell 29: 전체 추론 파이프라인 예제
"""
MemoryLLM의 전체 추론 파이프라인을 시연합니다.

흐름:
1. 컨텍스트 주입 (inject_memory)
2. 질문 생성 (generate)
3. 추가 컨텍스트 주입
4. 다시 질문 (지식 유지 확인)
"""

def run_inference_pipeline(model, tokenizer):
    """전체 추론 파이프라인을 실행합니다."""
    
    print("=== MemoryLLM 추론 파이프라인 ===\n")
    
    # 메모리 초기 상태 백업
    original_memory = model.memory.data.detach().cpu().clone()
    original_initialized = model.initialized.item()
    
    try:
        # 1. 첫 번째 컨텍스트 주입
        context1 = "John's favorite color is blue. He paints his room blue and drives a blue car."
        print(f"[1. 첫 번째 컨텍스트 주입]")
        print(f"  컨텍스트: '{context1}'")
        
        context1_ids = tokenizer(context1, return_tensors='pt').input_ids.to(model.device)
        model.inject_memory(context1_ids, update_memory=True)
        print(f"  ✓ 메모리 주입 완료\n")
        
        # 2. 첫 번째 질문
        question1 = "What is John's favorite color?"
        print(f"[2. 첫 번째 질문]")
        print(f"  질문: '{question1}'")
        
        q1_ids = tokenizer(question1, return_tensors='pt').input_ids.to(model.device)
        output1 = model.generate(
            inputs=q1_ids,
            max_new_tokens=20,
            pad_token_id=tokenizer.pad_token_id,
            do_sample=False,  # 결정적 생성
        )
        response1 = tokenizer.decode(output1[0], skip_special_tokens=True)
        print(f"  응답: '{response1}'\n")
        
        # 3. 두 번째 컨텍스트 주입 (무관한 정보)
        context2 = "Mary loves strawberries. She grows strawberries in her garden."
        print(f"[3. 두 번째 컨텍스트 주입 (무관한 정보)]")
        print(f"  컨텍스트: '{context2}'")
        
        context2_ids = tokenizer(context2, return_tensors='pt').input_ids.to(model.device)
        model.inject_memory(context2_ids, update_memory=True)
        print(f"  ✓ 메모리 주입 완료 (기존 메모리 일부 드롭)\n")
        
        # 4. 다시 첫 번째 질문 (지식 유지 확인)
        print(f"[4. 다시 첫 번째 질문 (지식 유지 확인)]")
        print(f"  질문: '{question1}'")
        
        output2 = model.generate(
            inputs=q1_ids,
            max_new_tokens=20,
            pad_token_id=tokenizer.pad_token_id,
            do_sample=False,
        )
        response2 = tokenizer.decode(output2[0], skip_special_tokens=True)
        print(f"  응답: '{response2}'\n")
        
        # 5. 결과 분석
        print(f"[5. 결과 분석]")
        print(f"  첫 번째 응답: '{response1}'")
        print(f"  두 번째 응답: '{response2}'")
        
        # 'blue'가 응답에 포함되는지 확인
        if 'blue' in response1.lower() and 'blue' in response2.lower():
            print(f"  ✓ 지식 유지 성공! 'blue' 정보가 유지됨")
        elif 'blue' in response1.lower():
            print(f"  △ 부분적 지식 유지: 첫 응답에만 'blue' 포함")
        else:
            print(f"  ✗ 지식 손실 가능성")
            
    finally:
        # 메모리 복원
        model.memory.data = original_memory.to(model.device)
        model.initialized.fill_(original_initialized)
        print(f"\n[메모리 원래 상태로 복원됨]")

# 실행 (모델이 로드된 경우에만)
if model is not None and tokenizer is not None:
    run_inference_pipeline(model, tokenizer)
else:
    print("모델이 로드되지 않았습니다.")
    print("\n위 코드는 다음 흐름을 시연합니다:")
    print("1. 컨텍스트 주입 → 질문 → 응답 확인")
    print("2. 추가 컨텍스트 주입 → 같은 질문 → 지식 유지 확인")

---

## Phase 4 완료!

이 섹션에서 다룬 내용:
- `forward()` 메서드 시그니처 및 MemoryLLM 추가 파라미터
- Forward pass 단계별 분석
- Cache position 계산 로직
- 텍스트 생성 (`generate()`) 사용법
- 전체 추론 파이프라인 예제

### Inference 핵심 정리

```
전체 Inference 흐름:

1. inject_memory(context_ids, update_memory=True)
   → delta_memory 추출 → 메모리 풀 업데이트

2. model.generate(question_ids)
   → forward(is_injection=False)
   → 전체 메모리 + 질문 연결
   → 토큰 생성
```

다음 섹션에서는 **Training Architecture**를 분석합니다.

---

# Part 5: Training Architecture

이 섹션에서는 MemoryLLM의 학습 아키텍처를 분석합니다.

## 5.1 PyTorch Lightning 구조

MemoryLLM의 학습은 **PyTorch Lightning** 프레임워크를 기반으로 합니다.

### 클래스 계층 구조

```
LlamaMemoryModelPL (train/MemoryLLM/memoryllm/models/memory.py)
    └── BaseMemoryModelPL (train/MemoryLLM/memoryllm/models/base.py)
            └── pl.LightningModule
                    └── LlamaDropMemoryModel (modules/memory_llama.py)
                            └── LlamaForCausalLM
                                    └── BaseMemoryModel
```

### 주요 파일

| 파일 | 역할 |
|------|------|
| `train/main.py` | 학습 엔트리 포인트, DataModule |
| `train/MemoryLLM/memoryllm/models/memory.py` | `LlamaMemoryModelPL` 정의 |
| `train/MemoryLLM/memoryllm/models/base.py` | `BaseMemoryModelPL` (training_step, validation_step) |
| `train/MemoryLLM/memoryllm/modules/memory_llama.py` | `LlamaDropMemoryModel` (학습용 모델) |
| `train/MemoryLLM/configs/` | YAML 설정 파일들 |

## 5.2 YAML 설정 파일 분석

학습 설정은 YAML 파일로 관리됩니다.

### 예시: `llama_30x256.yaml`

```yaml
model:
  base_learning_rate: 4.6e-6           # 학습률 (LoRA 사용 시 작은 값)
  target: MemoryLLM.memoryllm.models.memory.LlamaMemoryModelPL
  params:
    monitor: val/avg_acc               # 검증 지표
    num_blocks: 30                     # 메모리 블록 수
    num_tokens: 256                    # 블록당 토큰 수
    update_memory_during_training: true
    
    # LoRA 설정
    lora_config:
      r: 8                             # LoRA 랭크
      lora_alpha: 32
      lora_dropout: 0.1
      target_modules: ['q_proj', 'v_proj', 'k_proj', 'up_proj', 'down_proj', 'gate_proj']
    
    # 메모리 전략
    cat_and_drop_memory: true          # 핵심: 다중 컨텍스트 학습
    drop_memory_per_layer: true
    
    # 컨텍스트 스케줄
    num_contexts_schedule:
      checkpoints: [10000, 15000, 20000, 30000, 50000, 60000]
      values: [1, 2, 3, 4, 5, 10, 20]  # 점진적 컨텍스트 증가

data:
  batch_size: 1
  train:
    target: MemoryLLM.memoryllm.data.redpajama.RedPajamaDataset
  validation:
    - target: MemoryLLM.memoryllm.data.nq.NQDataset
    - target: MemoryLLM.memoryllm.data.squad.SQuADDataset

lightning:
  trainer:
    accelerator: gpu
    strategy: deepspeed_stage_2        # 분산 학습
    precision: 16-mixed                # 혼합 정밀도
    accumulate_grad_batches: 4         # 그래디언트 누적
```

## 5.3 training_step() 핵심 로직

`training_step()`은 MemoryLLM 학습의 핵심입니다.

### cat_and_drop_memory 전략

```python
def training_step(self, batch, batch_idx):
    contexts_ids, contexts_attention_masks, sentence_ids, \
    sentence_attention_masks, labels = batch
    
    # 현재 step에 맞는 컨텍스트 수 결정
    num_of_contexts = self.num_contexts(self.trainer.global_step)
    
    # === cat_and_drop_memory 전략 ===
    if self.cat_and_drop_memory:
        all_delta_memory = None
        
        # 여러 컨텍스트 순차 처리
        for i in range(len(contexts_ids)):
            output = self.model(
                input_ids=contexts_ids[i],
                attention_mask=contexts_attention_masks[i],
                output_delta_memory=True,
                is_injection=True
            )
            
            delta_memory = output.delta_memory.detach()
            
            if all_delta_memory is None:
                all_delta_memory = delta_memory
            else:
                # 기존 delta_memory에서 일부 드롭 + 새 delta_memory 추가
                all_delta_memory = self.model.drop_memory(all_delta_memory[0]).unsqueeze(0)
                all_delta_memory = torch.cat([all_delta_memory, delta_memory], dim=2)
        
        delta_memory = all_delta_memory.detach()
    
    # 문장에 대한 Language Modeling
    sentence_labels = sentence_ids.clone()
    sentence_labels[sentence_attention_masks == 0] = -100  # 패딩 무시
    
    output = self.model(
        input_ids=sentence_ids,
        attention_mask=sentence_attention_masks,
        labels=sentence_labels,
        delta_memory=delta_memory,
        is_injection=False
    )
    
    return output.loss
```

### num_contexts_schedule

컨텍스트 수를 점진적으로 증가시킵니다:

```
Step 0 ~ 10000:     1개 컨텍스트  (단순 학습)
Step 10000 ~ 15000: 2개 컨텍스트
Step 15000 ~ 20000: 3개 컨텍스트
...
Step 60000+:        20개 컨텍스트 (복잡한 메모리 관리 학습)
```

이 전략은 **커리큘럼 학습**의 일종으로, 모델이 점진적으로 복잡한 메모리 관리를 학습합니다.

In [ ]:
# Cell 33: 학습 실행 명령어
"""
MemoryLLM 학습 실행 방법입니다.

주의: 실제 학습에는 고사양 GPU (A100 80GB 권장)가 필요합니다.
"""

print("=== MemoryLLM 학습 실행 방법 ===\n")

print("[1. 환경 준비]")
print("  cd /path/to/MemoryLLM/train")
print("  pip install -r ../requirements.txt")
print()

print("[2. 데이터 준비]")
print("  # RedPajama 데이터 다운로드")
print("  mkdir -p data/redpajama")
print("  # (RedPajama 데이터는 별도 다운로드 필요)")
print()
print("  # NQ/SQuAD 검증 데이터")
print("  mkdir -p data/nq data/squad")
print("  # (데이터셋 다운로드)")
print()

print("[3. 학습 실행]")
print("  # Llama-2-7B 기반 학습 (30 blocks × 256 tokens)")
print("  python main.py -t --base MemoryLLM/configs/llama/llama_30x256.yaml")
print()
print("  # OpenLLaMA 디버깅용 (작은 설정)")
print("  python main.py -t --base MemoryLLM/configs/openllama/openllama_4x256.yaml")
print()

print("[4. 학습 모니터링]")
print("  # TensorBoard")
print("  tensorboard --logdir=logs/")
print()

print("[5. 체크포인트에서 재개]")
print("  python main.py -t --base MemoryLLM/configs/llama/llama_30x256.yaml \\")
print("                 -r /path/to/checkpoint.ckpt")
print()

print("[참고: 예상 학습 시간]")
print("  - A100 80GB 1장: ~2-3일 (60K steps)")
print("  - 메모리 요구량: ~40GB+ (DeepSpeed Stage 2)")
print("  - 그래디언트 누적: 4 (effective batch size = 4)")

---

## Phase 5 & 6 완료!

### Training 핵심 요약

| 항목 | 설명 |
|------|------|
| **프레임워크** | PyTorch Lightning + DeepSpeed |
| **학습 전략** | `cat_and_drop_memory` (다중 컨텍스트 처리) |
| **파라미터 효율화** | LoRA (r=8, alpha=32) |
| **커리큘럼 학습** | `num_contexts_schedule` (1→20 점진 증가) |
| **데이터셋** | RedPajama (학습), NQ/SQuAD (검증) |

---

# Part 7: Evaluation & Metrics

## 7.1 Knowledge Retention 평가

MemoryLLM의 핵심 평가는 **지식 유지 능력**입니다.

### 평가 시나리오

```
1. 관련 컨텍스트 주입 → 질문 → 정확도 측정 (Acc_0)
2. 무관한 컨텍스트 1개 추가 → 같은 질문 → 정확도 측정 (Acc_1)
3. 무관한 컨텍스트 2개 추가 → 같은 질문 → 정확도 측정 (Acc_2)
...
N. 무관한 컨텍스트 N개 추가 → 같은 질문 → 정확도 측정 (Acc_N)

목표: Acc_N ≈ Acc_0 (무관한 정보가 추가되어도 지식 유지)
```

### 평가 실행

```bash
mkdir results

# NQ + SQuAD 평가 (5개 무관한 컨텍스트)
python test_qa_memory.py \
    --model YuWangX/memoryllm-7b \
    --nuc 5 \
    --datasets naturalqa squad \
    --num_samples 100

# LongBench 평가
python longbench_pred.py \
    --model memoryllm-7b \
    --datasets hotpotqa \
    --max_length 16384
```

## 7.2 평가 메트릭

`metrics.py`에 정의된 주요 메트릭:

| 메트릭 | 함수 | 용도 |
|--------|------|------|
| **Exact Match** | `normalize_answer()` | QA 정확도 |
| **F1 Score** | `qa_f1_score()` | 토큰 레벨 F1 |
| **ROUGE-L** | `rouge_score()` | 요약 평가 |
| **Count Score** | `count_score()` | 숫자 추출 |

---

# Part 8: Advanced Topics

## 8.1 M+ (MPlus) 확장 모델

M+는 MemoryLLM에 **Long-Term Memory (LTM)**을 추가한 확장 버전입니다.

### MemoryLLM vs M+

| 특성 | MemoryLLM | M+ |
|------|-----------|------|
| **메모리 유형** | STM only | STM + LTM |
| **메모리 크기** | 고정 (12800 tokens) | 확장 가능 |
| **저장 위치** | GPU | STM: GPU, LTM: CPU |
| **검색 메커니즘** | 없음 | Modern Hopfield Network |

### M+ 사용법

```python
from modeling_mplus import MPlus

model = MPlus.from_pretrained("YuWangX/mplus-8b", 
                               torch_dtype=torch.float16,
                               device_map="auto")

# LTM 검색 활성화
model.inject_memory(context_ids, update_memory=True, use_retriever=True)
```

## 8.2 Chat Model 사용법

MemoryLLM은 Chat 버전도 제공합니다.

```python
model = MemoryLLM.from_pretrained("YuWangX/memoryllm-8b-chat", ...)
tokenizer = AutoTokenizer.from_pretrained("YuWangX/memoryllm-8b-chat")

# 컨텍스트 주입
ctx = "Last week, John had a wonderful picnic with David. David loves strawberries."
model.inject_memory(tokenizer(ctx, return_tensors='pt').input_ids.cuda(), update_memory=True)

# Chat 템플릿 적용
messages = [{"role": "user", "content": "What fruit does David like?"}]
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", 
                                        add_generation_prompt=True)[:, 1:]

# 생성
outputs = model.generate(
    inputs.cuda(), 
    max_new_tokens=50,
    eos_token_id=[tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]
)
```

## 8.3 Best Practices

### 메모리 관리

1. **메모리 백업**: 중요한 시점에 메모리 상태 저장
   ```python
   backup = model.memory.data.detach().cpu().clone()
   # ... 작업 수행 ...
   model.memory.data = backup.to(model.device)
   ```

2. **메모리 리셋**: 새 세션 시작 시
   ```python
   model.initialized.fill_(0)
   ```

3. **배치 처리 주의**: 현재 batch_size=1만 안정적으로 지원

### 성능 최적화

- **Flash Attention 2**: 추론 속도 향상
- **torch.float16**: 메모리 절약
- **gradient_checkpointing**: 학습 시 메모리 절약

---

# 노트북 완료!

## 전체 요약

이 노트북에서 다룬 내용:

### Part 1-4: Inference (추론)
- MemoryLLM 아키텍처 및 Memory Pool 구조
- `inject_memory()`, `update_memory_with_delta_memory()`, `drop_memory()`, `cat_memory_and_hiddens()`
- Forward pass 및 텍스트 생성

### Part 5-6: Training (학습)
- PyTorch Lightning + DeepSpeed 기반 학습
- `cat_and_drop_memory` 전략
- LoRA 파라미터 효율화
- 커리큘럼 학습 (`num_contexts_schedule`)

### Part 7-8: Evaluation & Advanced (평가 및 고급)
- Knowledge Retention 평가
- M+ (MPlus) 확장 모델
- Chat Model 사용법
- Best Practices

## Quick Reference

```python
# === Inference ===
from modeling_memoryllm import MemoryLLM

model = MemoryLLM.from_pretrained("YuWangX/memoryllm-8b", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("YuWangX/memoryllm-8b")

# 컨텍스트 주입
model.inject_memory(tokenizer(context, return_tensors='pt').input_ids.cuda(), update_memory=True)

# 생성
output = model.generate(tokenizer(question, return_tensors='pt').input_ids.cuda(), max_new_tokens=20)
print(tokenizer.decode(output[0]))

# === Training ===
cd train/
python main.py -t --base MemoryLLM/configs/llama/llama_30x256.yaml

# === Evaluation ===
python test_qa_memory.py --model YuWangX/memoryllm-7b --nuc 5 --datasets naturalqa squad
```

## 참고 자료

- **논문**: [MemoryLLM: Towards Self-Updatable Large Language Models](https://arxiv.org/abs/2402.04624) (ICML 2024)
- **GitHub**: [YuWangX/MemoryLLM](https://github.com/YuWangX/MemoryLLM)
- **HuggingFace**: [YuWangX/memoryllm-8b](https://huggingface.co/YuWangX/memoryllm-8b)